# 1. Load Dataset 🔄

In [30]:
import os
import numpy as np
import random
import csv
from typing import List, Tuple, Any
import math
import pickle

In [31]:
def load_dataset(file_path: str) -> Tuple[List[List[float]], List[Any]]:
    """
    Loads the dataset from a CSV file.

    This function assumes that the CSV file has a header row and that:
    - All columns except the last one are features (converted to float).
    - The last column is the label (left as a string; convert if needed).

    Note:
      The current implementation is particularly suited for datasets like the Iris dataset.
      For other datasets, you might want to modify the logic (e.g., to change the label column index).

    Parameters:
      file_path (str): Path to the dataset file.

    Returns:
      Tuple[List[List[float]], List[Any]]:
          - data: A list of rows, each row is a list of features as floats.
          - labels: A list of labels corresponding to each row.
    """
    data: List[List[float]] = []
    labels: List[Any] = []

    try:
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            header = next(reader, None)  # Skip header row if exists

            # Process each row in the CSV file
            for row in reader:
                if row:  # Ensure the row is not empty
                    # Convert all columns except the last one into floats (features)
                    try:
                        features = [float(item) for item in row[:-1]]
                    except ValueError as ve:
                        print(f"Could not convert features to float in row: {row}. Error: {ve}")
                        continue
                    # The last column is considered the label (remains as string or processed further)
                    label = row[-1]

                    data.append(features)
                    labels.append(label)

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

    return data, labels

In [37]:
    dataset, labels = load_dataset('/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv')
    print(dataset[:3],"\n")
    print(labels[:3])

[[1.0, 5.1, 3.5, 1.4, 0.2], [2.0, 4.9, 3.0, 1.4, 0.2], [3.0, 4.7, 3.2, 1.3, 0.2]] 

['Iris-setosa', 'Iris-setosa', 'Iris-setosa']


In [3]:
def validate_dataset(dataset: List) -> bool:
    """
    Validates the structure and integrity of the dataset.

    Parameters:
    dataset (list): The loaded dataset.

    Returns:
    bool: True if dataset is valid, False otherwise.
    """
    if len(dataset) == 0:
        return False

    num_features = len(dataset[0])
    for row in dataset:
        if len(row) != num_features:
            return False

    return True

In [25]:
def test_data_loading():
    """
    Tests if the dataset loading function works as expected.
    """
    try:
        file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'
        dataset, labels = load_dataset(file_path)
        assert validate_dataset(dataset), "Dataset validation failed."
        print("Data loaded and validated successfully.")
    except AssertionError as error:
        print(f"Test failed: {error}")
    except FileNotFoundError:
        print("File not found. Make sure the dataset file exists at the specified path.")
    except Exception as error:
        print(f"An error occurred: {error}")

# 2. Normalize the Data 🔄

In [5]:
def transpose(data: List[List]) -> List[List]:
    """Transposes a list of lists.

    Args:
    data: A list of lists.

    Returns:
    A transposed list of lists.
    """

    return list(zip(*data))

In [6]:
def min_max_normalizer(data: List[List[float]]) -> List[List[float]]:
    """
    Applies Min-Max scaling to the dataset, scaling features between 0 and 1.

    Parameters:
    data (list of lists): The dataset to be normalized.

    Returns:
    scaled_data (list of lists): Min-max normalized dataset.
    """

    transposed_data = transpose(data)

    min_vals = [min(col) for col in transposed_data]
    max_vals = [max(col) for col in transposed_data]

    scaled_data = []

    for row in data:
        try:
            scaled_row = [(val - min_val) / (max_val - min_val) if (max_val - min_val) != 0 else 0.0 for val, min_val,max_val in zip(row, min_vals, max_vals)]
            scaled_data.append(scaled_row)
        except ZeroDivisionError:
            row_index = data.index(row)
            print(f"ZeroDivisionError encountered in row {row_index}. Values: {row}, {min_vals}, {max_vals}")
            raise ZeroDivisionError

    return scaled_data

In [7]:
def split_data(dataset: List[List[float]], training_size: float = 0.7, validation_size: float = 0.15) -> Tuple[List[List[float]], List[List[float]], List[List[float]]]:
    """
    Splits the dataset into training, validation, and test sets.

    Parameters:
    dataset (list of lists): The dataset to be split.
    training_size (float): Proportion of data to be used for training.
    validation_size (float): Proportion of data to be used for validation.

    Returns:
    Tuple containing the training, validation, and test sets.
    """
    random.shuffle(dataset)

    total_size = len(dataset)
    train_end = int(training_size * total_size)
    val_end = int((training_size + validation_size) * total_size)

    training_data = dataset[:train_end]
    validation_data = dataset[train_end:val_end]
    test_data = dataset[val_end:]

    return training_data, validation_data, test_data

In [17]:
def test_split_data():
    """
    Tests if the data splitting function works as expected.
    """
    try:
        file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'
        dataset, labels = load_dataset(file_path)
        dataset = min_max_normalizer(dataset)
        training_data, validation_data, test_data = split_data(dataset)

        assert len(training_data) > 0, "Training data is empty."
        assert len(validation_data) > 0, "Validation data is empty."
        assert len(test_data) > 0, "Test data is empty."
        total_size = len(training_data) + len(validation_data) + len(test_data)
        assert total_size == len(dataset), "Data splitting error: sizes do not match."

        print("Data splitting test passed successfully.")
    except AssertionError as error:
        print(f"Test failed: {error}")
    except Exception as error:
        print(f"An error occurred: {error}")

Data splitting test passed successfully.


# 3. Define the Architecture 🏗

In [9]:
class Neuron:
    """A simple neuron class with type hints."""

    def __init__(self, weights: List[float], bias: float = None):
        """Initializes a neuron with given weights and bias.

        Args:
            weights: A list of weights for the neuron's inputs.
            bias: The bias term for the neuron.
        """
        self.weights = weights
        self.bias = bias if bias is not None else random.uniform(-0.1, 0.1)
        self.inputs = []
        self.output = 0

    def forward(self, inputs: List[float]) -> float:
        """Calculates the output of the neuron.

        Args:
            inputs: A list of inputs to the neuron.

        Returns:
            The output of the neuron.
        """
        self.inputs = inputs
        weighted_sum = sum([input_ * weight for input_, weight in zip(inputs, self.weights)])
        self.output = weighted_sum + self.bias
        return self.output

    def activation(self, output: float) -> float:
        """Applies an activation function to the neuron's output.

        Args:
            output: The output of the neuron.

        Returns:
            The activated output of the neuron.
        """
        return output

    def compute_gradient(self, delta: float) -> List[float]:
        """Calculates the gradient for the weights using the delta from the next layer.

        Args:
            delta: The error signal from the next layer.

        Returns:
            A list of gradients for the weights.
        """
        gradients = [delta * input_ for input_ in self.inputs]
        return gradients

    def update_weights(self, learning_rate: float, gradients: List[float]):
        """Updates the weights and bias of the neuron using the computed gradients.

        Args:
            learning_rate: The learning rate for weight updates.
            gradients: The computed gradients for each weight.
        """
        self.weights = [w - learning_rate * g for w, g in zip(self.weights, gradients)]
        self.bias -= learning_rate * gradients[-1]

    def propagate_error_back(self) -> List[float]:
        """Propagates the error signal back to the previous layer.

        Returns:
            A list of error terms to propagate to the previous layer.
        """
        error_signal = self.output * (1 - self.output)
        propagated_errors = [error_signal * weight for weight in self.weights]
        return propagated_errors

In [10]:
class ActivationFunctions:
    """A utility class for activation functions."""

    @staticmethod
    def sigmoid(x: float) -> float:
        """Applies the sigmoid activation function.

        Args:
            x: The input value.

        Returns:
            The activated value.
        """
        return 1 / (1 + math.exp(-x))

    @staticmethod
    def relu(x:float) -> float:
        """Applies the rectified linear unit (ReLU) activation function.

        Args:
            x: The input value.

        Returns:
            The activated value.
        """
        return max(0,x)

    @staticmethod
    def softmax(x: List[float]) -> List[float]:
        """Applies the softmax activation function.

        Args:
            x: A list of input values.

        Returns:
            A list of normalized probabilities.
        """

        exp_values = [math.exp(value) for value in x]
        sum_exp_values = sum(exp_values)
        return [exp_value / sum_exp_values for exp_value in exp_values]

In [11]:
class Layer:
    """A layer in a neural network."""

    def __init__(self, neurons: List[Neuron], activation: ActivationFunctions, is_output_layer: bool = False):
        """Initializes a layer with the specified number of neurons and inputs.

        Args:
            neurons: The neurons in the layer.
            activation: The activation function to use for the neurons.
        """
        self.neurons = neurons
        self.activation = activation
        self.is_output_layer = is_output_layer

    def forward(self, inputs: List[float]) -> List[float]:
        """Propagates inputs through the layer.

        Args:
            inputs: A list of inputs to the layer.

        Returns:
            A list of outputs from the neurons in the layer.
        """
        if self.is_output_layer:
            logits = [neuron.forward(inputs) for neuron in self.neurons]
            return ActivationFunctions.softmax(logits)
        else:
            neuron_outputs = [neuron.forward(inputs) for neuron in self.neurons]
            activated_outputs = [self.activation(output) for output in neuron_outputs]
            return activated_outputs

    def backward(self, delta: List[float], learning_rate: float) -> List[float]:
        """Performs the backward pass and updates weights and biases.

        Args:
            delta: The error term (gradient) from the next layer.
            learning_rate: The learning rate for weight updates.

        Returns:
            A list of propagated error terms to pass to the previous layer.
        """
        new_delta = []
        for i, neuron in enumerate(self.neurons):
            neuron_gradient = neuron.compute_gradient(delta[i])

            neuron.update_weights(learning_rate)

            new_delta.append(neuron.propagate_error_back())

        return new_delta

In [12]:
class Network:
    """A neural network with multiple layers."""


    def __init__(self, layers: List[Layer], epochs: int, learning_rate: float):
        self.layers = layers
        self.learning_rate = learning_rate
        self.epochs = epochs

    def forward(self, inputs: List[float]) -> List[float]:
        """Propagates inputs through the entire network.

        Args:
            inputs: A list of inputs to the network.

        Returns:
            A list of outputs from the final layer.
        """
        outputs = inputs
        for layer in self.layers:
            outputs = layer.forward(outputs)
        return outputs

    def backward(self, targets: List[float], outputs: List[float]):
        """Performs backpropagation to update the network's weights and biases.

        Args:
            targets: A list of target outputs.
            outputs: A list of outputs from the final forward pass.
        """
        delta = self.loss_derivative(outputs, targets)

        for layer in reversed(self.layers):
            delta = layer.backward(delta, self.learning_rate)

    def compute_loss(self, predicted: List[float], actual: List[float]) -> float:
        """Calculates the loss for the predictions."""
        return LossFunction.cross_entropy(predicted, actual)

    def loss_derivative(self, outputs: List[float], targets: List[float]) -> List[float]:
        """Computes the derivative of the loss function."""
        return [pred - target for pred, target in zip(outputs, targets)]

    def train(self, training_data: List[tuple]):
        """Trains the network on the given data without mini-batches.

        Args:
            training_data: A list of tuples containing input and target data.
        """
        num_samples = len(training_data)

        for epoch in range(self.epochs):
            total_loss = 0
            random.shuffle(training_data)

            for inputs, targets in training_data:
                outputs = self.forward(inputs)

                loss = self.compute_loss(outputs, targets)
                total_loss += loss

                self.backward(targets, outputs)

            avg_loss = total_loss / num_samples

            print(f"Epoch {epoch + 1}/{self.epochs} complete. Average loss: {avg_loss:.4f}")

    def evaluate(self, test_data: List[tuple]) -> float:
        """Evaluates the network on the test data."""
        inputs_batch, targets_batch = zip(*test_data)
        predictions = [self.forward(inputs) for inputs in inputs_batch]
        accuracy = self.calculate_accuracy(predictions, targets_batch)
        return accuracy

    def predict(self, new_data: List[float]) -> List[float]:
        """Predicts the output for new input data.

        Args:
            new_data: A list of new input data.

        Returns:
            A list of predicted outputs.
        """
        return self.forward(new_data)

    def calculate_accuracy(self, predictions: List[List[float]], targets: List[List[float]]) -> float:
        """Calculates the accuracy of the model."""
        correct_predictions = 0
        for pred, target in zip(predictions, targets):
            predicted_class = np.argmax(pred)
            true_class = np.argmax(target)
            if predicted_class == true_class:
                correct_predictions += 1
        return correct_predictions / len(targets)

    def save_weights(self, filename: str):
        """Saves the weights of the network to a file."""
        weights = [[neuron.weights for neuron in layer.neurons] for layer in self.layers]
        biases = [[neuron.bias for neuron in layer.neurons] for layer in self.layers]

        with open(filename, 'wb') as f:
            pickle.dump((weights, biases), f)

    def load_weights(self, filename: str):
        """Loads weights into the network from a file."""
        with open(filename, 'rb') as f:
            weights, biases = pickle.load(f)

        for layer, layer_weights, layer_biases in zip(self.layers, weights, biases):
            for neuron, neuron_weights, neuron_bias in zip(layer.neurons, layer_weights, layer_biases):
                neuron.weights = neuron_weights
                neuron.bias = neuron_bias

In [13]:
class LossFunction:
    """A utility class for loss functions."""

    @staticmethod
    def cross_entropy(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """Calculates the categorical cross-entropy loss.

        Args:
            predicted_outputs: A list of predicted probabilities for each class.
            actual_outputs: A one-hot encoded list of actual output values.

        Returns:
            The categorical cross-entropy loss.
        """
        # Clip the predicted values to prevent log(0)
        predicted_outputs = np.clip(predicted_outputs, 1e-12, 1 - 1e-12)

        # Calculate the cross-entropy loss
        loss = -sum([actual_output * np.log(predicted_output)
                     for predicted_output, actual_output in zip(predicted_outputs, actual_outputs)])
        return loss / len(predicted_outputs)

    @staticmethod
    def mean_squared_error(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """Calculates the mean squared error (MSE).

        Args:
            predicted_outputs: A list of predicted output values.
            actual_outputs: A list of actual output values.

        Returns:
            The mean squared error.
        """
        squared_errors = [(predicted_output - actual_output) ** 2
                          for predicted_output, actual_output in zip(predicted_outputs, actual_outputs)]
        return sum(squared_errors) / len(squared_errors)


In [14]:
class WeghitsInitializer:
    """A utility class for weight initialization."""

    @staticmethod
    def weights(num_inputs: int) -> List[float]:
        """Initializes weights with random values uniformly distributed between -0.1 and 0.1.

        Args:
            num_inputs: The number of inputs to the neuron.

        Returns:
            A list of initialized weights.
        """
        return [random.uniform(-0.1, 0.1) for _ in range(num_inputs)]


# 4. Workflow 🔮: Load Dataset

In [18]:
test_data_loading()
file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'
dataset, labels = load_dataset(file_path)
test_split_data()
training_data, validation_data, test_data = split_data(dataset)
print(training_data)
print(validation_data)
print(test_data)

Data loaded and validated successfully.
Data splitting test passed successfully.
[[38.0, 4.9, 3.1, 1.5, 0.1], [89.0, 5.6, 3.0, 4.1, 1.3], [141.0, 6.7, 3.1, 5.6, 2.4], [4.0, 4.6, 3.1, 1.5, 0.2], [99.0, 5.1, 2.5, 3.0, 1.1], [50.0, 5.0, 3.3, 1.4, 0.2], [111.0, 6.5, 3.2, 5.1, 2.0], [9.0, 4.4, 2.9, 1.4, 0.2], [91.0, 5.5, 2.6, 4.4, 1.2], [123.0, 7.7, 2.8, 6.7, 2.0], [109.0, 6.7, 2.5, 5.8, 1.8], [19.0, 5.7, 3.8, 1.7, 0.3], [24.0, 5.1, 3.3, 1.7, 0.5], [31.0, 4.8, 3.1, 1.6, 0.2], [86.0, 6.0, 3.4, 4.5, 1.6], [132.0, 7.9, 3.8, 6.4, 2.0], [112.0, 6.4, 2.7, 5.3, 1.9], [5.0, 5.0, 3.6, 1.4, 0.2], [15.0, 5.8, 4.0, 1.2, 0.2], [101.0, 6.3, 3.3, 6.0, 2.5], [80.0, 5.7, 2.6, 3.5, 1.0], [57.0, 6.3, 3.3, 4.7, 1.6], [118.0, 7.7, 3.8, 6.7, 2.2], [18.0, 5.1, 3.5, 1.4, 0.3], [126.0, 7.2, 3.2, 6.0, 1.8], [149.0, 6.2, 3.4, 5.4, 2.3], [150.0, 5.9, 3.0, 5.1, 1.8], [27.0, 5.0, 3.4, 1.6, 0.4], [66.0, 6.7, 3.1, 4.4, 1.4], [131.0, 7.4, 2.8, 6.1, 1.9], [114.0, 5.7, 2.5, 5.0, 2.0], [63.0, 6.0, 2.2, 4.0, 1.0], [55.0, 6.5, 

# Workflow 🔮: Architecture

## Create input layer

In [19]:
weights = WeghitsInitializer.weights(4)
bias = random.uniform(-0.1, 0.1)
input_neurons = [Neuron(weights=weights, bias=bias) for _ in range(4)]
input_layer = Layer(neurons=input_neurons, activation= ActivationFunctions.relu, is_output_layer=False)

# Print weights of input neurons
for neuron in input_layer.neurons:
    print("Input Neuron Weights:", neuron.weights)

Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]
Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]
Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]
Input Neuron Weights: [-0.012236743985385504, -0.07112458581385527, -0.02374207543011911, 0.09096790851691727]


## Create hidden layer

In [20]:
weights = WeghitsInitializer.weights(4)
bias = random.uniform(-0.1, 0.1)
hidden_neurons = [Neuron(weights=weights, bias=bias) for _ in range(5)]
hidden_layer = Layer(neurons=hidden_neurons, activation= ActivationFunctions.relu, is_output_layer=False)

# Print weights of input neurons
for neuron in hidden_layer.neurons:
    print("Input Neuron Weights:", neuron.weights)

Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]
Input Neuron Weights: [-0.0025985139119916456, 0.08967536608302626, 0.024991896165996758, -0.028760546744581533]


## Create output layer

In [21]:
weights = WeghitsInitializer.weights(5)
bias = random.uniform(-0.1, 0.1)
output_neurons = [Neuron(weights=weights, bias=bias) for _ in range(3)]
output_layer = Layer(neurons=output_neurons, activation= ActivationFunctions.relu, is_output_layer=True)

# Print weights of input neurons
for neuron in output_layer.neurons:
    print("Input Neuron Weights:", neuron.weights)

Input Neuron Weights: [0.061455682311839954, -0.013101149463906475, 0.09484125191639023, 0.04079782617445696, -0.049328948326537274]
Input Neuron Weights: [0.061455682311839954, -0.013101149463906475, 0.09484125191639023, 0.04079782617445696, -0.049328948326537274]
Input Neuron Weights: [0.061455682311839954, -0.013101149463906475, 0.09484125191639023, 0.04079782617445696, -0.049328948326537274]


## Train

In [22]:
# Define network architecture
epochs = 100
learning_rate = 0.01
layers = [input_layer, hidden_layer, output_layer]

# Initialize the network
network = Network(layers=layers, epochs=epochs, learning_rate=learning_rate)

num_samples = len(training_data)
print(num_samples)
print(training_data)
print(labels)

def match_and_combine(upper_list: List[List[float]], lower_list: List[List[float]]) -> List[Tuple[List[float], str]]:
    # Create a dictionary from the lower list
    lower_dict = {item[0]: item[1] for item in lower_list}

    # Create a list to store the result tuples
    result = []

    # Iterate through the upper list
    for item in upper_list:
        key = item[0]
        # Check if the key exists in the lower dictionary
        if key in lower_dict:
            # Create a tuple (without the key) and add to the result list
            result.append((item[1:], lower_dict[key]))

    return result

# Example usage
combined = match_and_combine(training_data, labels)
print(combined)

network.train(combined)


105
[[38.0, 4.9, 3.1, 1.5, 0.1], [89.0, 5.6, 3.0, 4.1, 1.3], [141.0, 6.7, 3.1, 5.6, 2.4], [4.0, 4.6, 3.1, 1.5, 0.2], [99.0, 5.1, 2.5, 3.0, 1.1], [50.0, 5.0, 3.3, 1.4, 0.2], [111.0, 6.5, 3.2, 5.1, 2.0], [9.0, 4.4, 2.9, 1.4, 0.2], [91.0, 5.5, 2.6, 4.4, 1.2], [123.0, 7.7, 2.8, 6.7, 2.0], [109.0, 6.7, 2.5, 5.8, 1.8], [19.0, 5.7, 3.8, 1.7, 0.3], [24.0, 5.1, 3.3, 1.7, 0.5], [31.0, 4.8, 3.1, 1.6, 0.2], [86.0, 6.0, 3.4, 4.5, 1.6], [132.0, 7.9, 3.8, 6.4, 2.0], [112.0, 6.4, 2.7, 5.3, 1.9], [5.0, 5.0, 3.6, 1.4, 0.2], [15.0, 5.8, 4.0, 1.2, 0.2], [101.0, 6.3, 3.3, 6.0, 2.5], [80.0, 5.7, 2.6, 3.5, 1.0], [57.0, 6.3, 3.3, 4.7, 1.6], [118.0, 7.7, 3.8, 6.7, 2.2], [18.0, 5.1, 3.5, 1.4, 0.3], [126.0, 7.2, 3.2, 6.0, 1.8], [149.0, 6.2, 3.4, 5.4, 2.3], [150.0, 5.9, 3.0, 5.1, 1.8], [27.0, 5.0, 3.4, 1.6, 0.4], [66.0, 6.7, 3.1, 4.4, 1.4], [131.0, 7.4, 2.8, 6.1, 1.9], [114.0, 5.7, 2.5, 5.0, 2.0], [63.0, 6.0, 2.2, 4.0, 1.0], [55.0, 6.5, 2.8, 4.6, 1.5], [72.0, 6.1, 2.8, 4.0, 1.3], [3.0, 4.7, 3.2, 1.3, 0.2], [106.0

TypeError: can't multiply sequence by non-int of type 'numpy.float64'